<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/main/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>